In [ ]:
!pip install peft transformers accelerate


In [ ]:
lora_path = "your-lora-adapter-path-or-repo"


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig

model_name = "Qwen/Qwen2.5-3B-Instruct"
lora_path = "your-lora-adapter-path-or-repo"  

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load base model
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

model = PeftModel.from_pretrained(base_model, lora_path)

# model = model.merge_and_unload()

prompt = "Berikan saya hukum hukum perdata indonesia"
messages = [
    {"role": "system", "content": "Kamu adalah AI tentang hukum Indonesia"},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)


generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)


generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]


response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)
